In [ ]:
intents = {"intents": [
    {"tag": "greetings",
    "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up?", "How is it going?", "Hey"],
    "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?", "Hello! What can I do for you?"]
    },
    {"tag": "goodbye",
    "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time", "Have a great day!", "I'm leaving", "Have a good day", "cao", "cya"],
    "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"]
    },
    {"tag": "age",
    "patterns": ["How old are you?", "What is your age?", "How many years have you?", "When were you born?", "What is your birth date?", "How old is Tejasvin?", "age?"],
    "responses": ["I am 1 month old!", "I was born in July 2024!", "My birthday is on 1st July 2024!", "I am 1 month old!", "My owner Tejasvin is 21 years old!"]
    },
    {"tag": "name",
    "patterns": ["What is your name?", "What should I call you?", "Who are you?", "name?", "Can you tell me your name?"],
    "responses": ["You can call me Tutu!", "I'm Tutu, version 1.0", "You can call me Tejasvin's Assistant.", "I'm Tejasvin's Assistant!", "I'm Tutu, your virtual assistant!"]
    },
    {"tag": "shop",
     "patterns": ["I want to buy something", "What do you have for sale?", "What is available for purchase?", "What are you selling?", "What is in stock?", "What do you have in your shop?"],
     "responses": ["We sell snacks, like samosas, chaat, burgers, pizza, pastries and much more", "We have snacks, like samosas, chaat, burgers, pizza, pastries", "We sell snacks, like samosas, chaat, burgers, pizza, pastries and much more, You can find them in the shop section."]
    },
    {"tag": "hours",
    "patterns": ["What are your hours?", "When are you open?", "What are your hours of operation?", "hours?"],
    "responses": ["We are open 7 days a week, 24 hours a day!", "Our hours are 24/7!", "We are open all day and night!"]
    },
    {"tag": "location",
    "patterns": ["Where are you located?", "What is your address?", "Where is your company situated?", "Where is your shop?", "location?"],
    "responses": ["We are located at 1234 Oppanakara St, Coimbatore, IN.", "Our Address is 1234 Oppanakara St, Coimbatore, IN.", "We are situated at 1234 Oppanakara St, Coimbatore, IN."]
    },
    {"tag": "payments",
    "patterns": ["Do you take credit cards?", "Do you accept Mastercard?", "Can I pay with Paypal?", "Are you cash only?", "payments?"],
    "responses": ["We accept VISA, Mastercard, and Paypal.", "We accept most major credit cards and Paypal.", "We accept VISA, Mastercard, and Paypal."]
    },
    {"tag": "funny",
    "patterns": ["Tell me a joke!", "Tell me something funny!", "Do you know a joke?", "funny?"],
    "responses": ["Why did the scarecrow win an award? Because he was outstanding in his field!", "Why did the tomato turn red? Because it saw the salad dressing!", "What do you call a fish with no eyes? A fsh!"]
    },
    {"tag": "thanks",
    "patterns": ["Thank you", "Thanks", "Thank you very much", "Thanks a lot!", "thanks?"],
    "responses": ["You're welcome!", "Anytime!", "My pleasure!", "Glad I could help!"]
    }
]}

In [ ]:
!pip install tensorflow
!pip install nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Split statements into words
        word_list = nltk.word_tokenize(pattern)
        #Appending context to list
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
      bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object) # Use dtype=object to handle lists of different lengths

train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [ ]:
# Use the legacy optimizer to support the 'decay' argument
from tensorflow.keras.optimizers.legacy import SGD
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print('Done')

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
12/12 [==============================] - 1s 3ms/step - loss: 2.3367 - accuracy: 0.0847
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 2.2490 - accuracy: 0.1525
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 2.2895 - accuracy: 0.1695
Epoch 4/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1929 - accuracy: 0.2203
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1623 - accuracy: 0.2034
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 2.0924 - accuracy: 0.2203
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 2.0025 - accuracy: 0.3220
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 1.9992 - accuracy: 0.3729
Epoch 9/200
12/12 [==============================] - 0s 3ms/step - loss: 1.8678 - accuracy: 0.4407
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 1.8659 - accuracy: 0.3729
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words) :
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [ ]:
from tkinter.messagebox import Message
print("GO! Bot is running!")

while True:
  message = input("You: ")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)

GO! Bot is running!
You: Hello
1/1 [==============================] - 0s 124ms/step
Hello!
You: Who are you?
1/1 [==============================] - 0s 18ms/step
I'm Tejasvin's Assistant!
You: How old are you?
1/1 [==============================] - 0s 19ms/step
My owner Tejasvin is 21 years old!
You: What all do you offer
1/1 [==============================] - 0s 18ms/step
We accept VISA, Mastercard, and Paypal.
You: What do you have in store
1/1 [==============================] - 0s 18ms/step
We sell snacks, like samosas, chaat, burgers, pizza, pastries and much more
You: bye
1/1 [==============================] - 0s 18ms/step
Talk to you later
You: Bye
1/1 [==============================] - 0s 18ms/step
Hi there, how can I help?
You: bye
1/1 [==============================] - 0s 18ms/step
Talk to you later
You: bye
1/1 [==============================] - 0s 17ms/step
Goodbye!


KeyboardInterrupt: Interrupted by user